# 参数初始化

现在我们已经知道如何访问参数，
让我们来看看如何正确地初始化它们。
我们在:numref:`sec_numerical_stability`中讨论了正确初始化的必要性。
深度学习框架为其层提供了默认的随机初始化。
然而，我们经常希望根据各种其他协议来初始化我们的权重。
该框架提供了最常用的协议，并且也允许创建自定义初始化器。

In [1]:
import torch
from torch import nn

默认情况下，PyTorch 通过从根据输入和输出维度计算的范围内均匀抽取来初始化权重和偏置矩阵。
PyTorch 的 `nn.init` 模块提供了多种预设的初始化方法。

In [2]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数
初始化为标准差为0.01的高斯随机变量，
而偏置参数则被清零。

In [3]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0128, -0.0103,  0.0103,  0.0009]), tensor(0.))

我们也可以将所有参数初始化为一个给定的常数值（比如，1）。

In [4]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

[**我们也可以为某些块应用不同的初始化器。**]
例如，下面我们将第一层
用Xavier初始化器进行初始化
并将第二层
初始化为常数值42。

In [5]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5314, -0.5090, -0.6871,  0.4311])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**自定义初始化**]

有时，我们需要的初始化方法
并没有被深度学习框架提供。
在下面的例子中，我们为任何权重参数 $w$ 定义了一个初始化器
使用以下奇怪的分布：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \textrm{ 以概率 } \frac{1}{4} \\
            0    & \textrm{ 以概率 } \frac{1}{2} \\
        U(-10, -5) & \textrm{ 以概率 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

同样，我们实现一个`my_init`函数来应用于`net`。

In [6]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  6.7383,  7.3089,  0.0000],
        [ 9.5873,  0.0000, -9.2901, -0.0000]], grad_fn=<SliceBackward0>)

请注意，我们始终可以选择直接设置参数。

In [7]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  7.7383,  8.3089,  1.0000])

## 摘要

我们可以使用内置和自定义初始化器来初始化参数。

## 练习

查阅在线文档以了解更多的内置初始化器。

[讨论](https://discuss.d2l.ai/t/8090)